# <div align="center">Skeletons</div>
---------------------------------------------------------------------

you can Find me on Github:
> ###### [ GitHub](https://github.com/lev1khachatryan)


A character’s ***skeleton*** is a pose-able framework of bones connected by articulated joints, arranged in a tree data structure. The skeleton itself is generally not rendered, but instead can be used as an invisible armature to position and orient render-able geometry such as a character’s skin, as we will see in later.

The ***joints*** allow relative movement within the skeleton, and they are represented mathematically by 4x4 linear transformation matrices. By combining the rotations, translations, scales, and shears possible with these matrices, a variety of joint types can be constructed, including hinges, ball-and-socket joints, sliding joints, and various other custom types. In practice, however, many character skeletons can be set up using only simple rotational joints, as they can adequately model the joints of most real animals.

Every joint has one or more degrees of freedom (***DoF***), which define its possible range of motion. For example, an elbow joint has one rotational DOF as it can only rotate along a single axis, while a shoulder joint has three DOFs, as it can rotate along three perpendicular axes. Individual joints usually have between one and six DOFs, but all together, a detailed character may have more than a hundred DOFs in the entire skeleton. Specifying values for these DOFs poses the skeleton, and changing these values over time results in movement, and is the essence of the animation process.

Given a set of specified DOF values, a ***joint local matrix*** can be constructed for each joint. These matrices define the position and orientation of each joint relative to the joint above it in the tree hierarchy. The local matrices can then be used to compute the ***world space matrices*** for all of the joints using the process of ***forward kinematics***. These world space matrices are what ultimately place the virtual character into the world, and can be used for skinning, rendering, collision detection, or other purposes.

In this notebook, we will examine the internal workings of the virtual skeleton. Section 1 discusses the details of forward kinematics and how it is applied to skeletons, starting with a brief review of some 3D geometry and linear algebra topics. Section 2 presents a variety of specific joint types that can be used in a character, as well as the matrix construction needed for these joints and Section 3 introduces the concept of a pose.

# 1. Forward Kinematics

The term ***kinematics*** refers to the mathematical description of motion without considering the underlying physical forces. Kinematics deals primarily with ***positions***, ***velocities***, ***accelerations***, and their ***rotational counterparts: orientation, angular velocity, and angular acceleration***. In this notebook, we are simply considered with computing static poses for skeletons and so we will limit our analysis mainly to ***positions and orientations***.

Two useful kinematic analysis tools are ***forward kinematics*** and ***inverse kinematics***. Within the scope of character animation, forward kinematics refers to the process of computing world space joint matrices based on specified DOF values, whereas inverse kinematics refers to the opposite problem of computing a set of DOF values that position a joint at a desired world space goal. Both forward and inverse kinematics are used in other fields such as robotics and mechanical engineering, and there is extensive literature available on the subject.

## 1.1 Basic Kinematics

This section presents a review of some basic linear algebra and is intended mainly as an introduction to the notation and standards. 

### coordinate system

Before delving deeper into the subject of character animation, we must first make a few basic definitions about ***coordinate systems***. We will use a three dimensional, right handed coordinate system by convention, meaning that the z-axis is the positive cross product of the x and y axes, with x pointing to the right, y pointing up, and z pointing to the viewer.

<img src='_assets/Prerequisites/1.png'>

Because the positive z-axis points outward, the viewer therefore looks in the –z direction. To be consistent with this coordinate system methodology, a character in a ‘default’ orientation would be aligned with the viewer, and would therefore look in the –z direction as well. Lights, cameras, vehicles, and other objects that get positioned with matrices will all be assumed to be facing down the –z axis in their default orientation.

Historically, different software and hardware rendering systems have disagreed upon the choice of coordinate systems and many different standards exist. The use of a right-handed system with the positive z-axis facing the viewer is probably the most widely accepted of these standards within the computer graphics industry and so it will be used here.

### homogeneous space

It is common practice in computer graphics to perform vector computations using ***4D homogeneous space***. Doing so allows various different operations (such as rotations and translations) to be combined into a single methodology.

<img src='_assets/Prerequisites/2.png'>

### Matrix Format

The 4x4 homogeneous matrix is a useful tool in computer graphics due to its ability to represent both the ***position and orientation*** of an object in space. Matrices can transform geometric data from one space to another and they are used extensively throughout character animation for a variety of purposes. To be consistent with most graphics texts, we choose to define the matrices with the translation along the bottom row, instead of along the right column as in many engineering texts. The right hand column is mainly used for viewing projections and is rarely needed for character animation. In almost every 4x4 matrix used in this book, the right hand column will contain three 0’s starting from the top and a 1 at the bottom. Matrices will generally take the following format: 

<img src='_assets/Prerequisites/3.png'>

where ***a, b, and c*** are the three basis vectors defining the ***orientation*** of the matrix and ***d*** is the ***position***. Usually, the three basis vectors will be of unit length and will be perpendicular to each other, making the matrix orthonormal or rigid, but this is not a strict requirement and some matrices may break that convention.

<img src='_assets/Prerequisites/4.png'>

### World and Local Space

In 3D graphics and animation, we define a fixed coordinate system called the world coordinate system or simply world space, in which all objects, characters, effects, cameras, lights and other entities are ultimately placed. The terms global coordinate system and global space are also commonly used and mean exactly the same thing, but for consistency, we will stick with the use of the word world rather than global.

Individual objects are typically defined in their own local space and make use of 4x4 matrices to transform to world space.

In a typical interactive graphics application, many different objects will need to co-exist in world space. Some of these objects are simple rigid objects, like a chair, for example. To manipulate the position and orientation of a simple object like this, we could use a single 4x4 matrix to transform the chair’s vertices from its local space to world space. This matrix is called the chair’s ***world matrix***, as it positions the chair into the world.

A more complex object, such as a character, will have many moving parts, and so will require many matrices. In order to render the character in the world and perform other operations such as collision detection, we need to know the world space matrices of all of the joints in the character’s skeleton. It is difficult and unintuitive to specify character joint matrices directly in world space, and so skeletons are built up of a hierarchy of local transformations, each defined relative to the one above it. The joint matrices are defined in this local space and are converted to world space by the process of ***forward kinematics***, described below.

### Cameras and View Space

To render a view of the 3D world, we place a virtual camera with a matrix called a ***camera matrix***. The space representing what the camera sees is called view space and objects are transformed into this space by the view matrix, which is the inverse of the camera matrix.

## 1.2 Joint Hierarchy

The topology of a skeleton is an open directed graph, or tree (also called a hierarchy). One joint is selected as the root and the other joints are connected up in hierarchical fashion.

To keep the definition of a skeleton simple, we will restrict them to being open trees without any closed loops. This restriction doesn’t really prevent kinematic loops in the final animated character, as we will learn about in.

The nodes in the tree represent the joints of the skeleton. They could just as easily represent the bones, and in fact, there is little difference between the concept of a bone and a joint, as the motion of a particular bone is the same as the motion of the joint controlling it. In this notebook, the two will be treated as the same thing, i.e., we may occasionally refer to a joint such as the shoulder in the exact same way we would refer to the bone directly manipulated by that joint (in this case, the upper arm or humerus bone). For consistency, we will usually describe things in terms of joints unless the situation specifically warrants the use of bones.

The following figure shows an example skeleton for a simple robot character. The hierarchical structure of the same skeleton is shown in figure below, with the root located at the top of the figure.

<img src='_assets/Prerequisites/5.png'>

<img src='_assets/Prerequisites/6.png'>

### Root Node

The choice of which node to make the root is somewhat arbitrary, but it is usually convenient to select something near the center of the character. A common choice on animals is somewhere in the spine, so that both the pelvis and torso can be attached underneath in the tree.

The root can be treated as a special joint that capable of unrestricted rotational and translational movement. In most characters, all other joints would have restrictions on their motion.

### Node Relationships

A node directly above another in the tree is that node’s parent. All nodes will have exactly one parent except for the root node, which has none. A node directly below another in the tree is that node’s child, and a node may have zero or more children. Child nodes inherit transformations from their parent nodes, so that if an elbow is rotated, for example, all of the joints in the hand will follow correctly. Nodes at the same level under a common parent are called siblings.

<img src='_assets/Prerequisites/7.png'>

A child of a child, (etc.) is called a descendant and a parent of a parent (etc.) is called an ancestor. Nodes with no children are called leaf nodes and nodes with children are called interior nodes.

It is said that a joint down in the tree inherits its transformation from its ancestors, that is, its own transformation builds on the ones that came above it. This concept can also be applied to other properties, such as rendering materials, or other visual properties, but we will not consider any of these other types of inheritance here.

The inheritance of the linear transformation information is handled through the process of forward kinematics and relies specifically on matrix concatenation.

### Linearization of the Hierarchy



## 1.3 Skeleton Kinematics

### Joint DoFs

A movable joint has one or more degrees of freedom, but typically they won’t have more than three. A free rigid body has 6 DoFs (3 to describe its position and 3 more to describe its rotation), but there isn’t really any reason why a joint couldn’t have 6 or even more DOFs. The root joint of a skeleton can be treated as a 6-DOF joint in most cases, unless the skeleton is somehow constrained to a fixed coordinate system.

The term ***degree of freedom*** is a general term that includes not only joint angles, but also joint translations, scales, or any other types of motion a joint may allow.

As DOFs can represent different types of quantities, it is important to keep track of the units used. For example, a rotational DoF could use degrees, radians, or any other arbitrary unit, as long as it is used consistently.

### Joint Local Matrix

A joint must take the input DoF values and use them to generate a joint local matrix. This matrix is a 4x4 homogeneous transformation matrix that defines the joint’s current position and orientation relative to its parent joint. Different types of joints will use different methods for generating this matrix.

### Joint Orientation

Some 3D animation systems allow a full fixed transformation to apply to the joint. The use of a full transformation means that we must apply a matrix multiplication to compute the complete joint local matrix instead of simply adding a translation to the bottom line. The purpose of this full transformation is to allow joints to rotate or translate around arbitrary axes, but as we will see throughout section 2.2, there are other straightforward ways to achieve this. Still, a full fixed orientation change can be supported for individual joints if desired. We will avoid this extra matrix, however, as we prefer other means for achieving the same results.

### Joint Limits

Joint DOFs can have limits on their range of movement. For example, the human elbow can bend to about +150 degrees (about 2.1 radians) and hyperextend back as much as –10 degrees (about -.17 radians). Limits should be able to be set on a DOF-by-DOF basis. In practice, it is common to have minimum and maximum limits for each DOF that can be enabled or disabled independently.

### Matrix Concatenation

Concatenating the local matrices to make the world space matrices is straightforward and makes use of matrix algebra and the very useful properties of 4x4 homogeneous matrices. To compute all of the world space matrices for a skeleton, we begin at the root and perform a depth-first tree traversal. For each joint visited in the traversal, we compute its world space matrix W joint by multiplying its local matrix L joint by its parent’s world space matrix W parent:

<img src='_assets/Prerequisites/8.png'>

The root node has no parent, and so Wparent is just the identity matrix, which causes its world space matrix to be equal to its joint local matrix.

Many modern CPU and graphics processors are equipped with vector floating point units that are designed specifically to handle 4x4 matrix concatenation and similar computations. Taking advantage of features like these should result in significant performance gains.

### Skeleton Forward Kinematics Algorithm

The end result of the forward kinematics process for a skeleton is a set of world space matrices- one for each joint. If we assume, for now, that the character is posed by some higher level system and its joint DOF values are all specified, then the two main computational steps needed per joint to compute the world space matrices are:

1. Generate joint local matrix
2. Concatenate joint local matrix to compute world space matrix